# 6.2 Synchrotron and Synchrotron Self Compton

In this tutorial, we will show how to compute the Spectral Energy Distribution (SED) produced by the Synchrotron and Synchrotron Self-Compton radiative processes.

We will also validate the results produced by `agnpy` by comparing them against the literature.

In [ ]:
# import numpy, astropy and matplotlib for basic functionalities
import numpy as np
import astropy.units as u
from astropy.constants import m_e, m_p
from astropy.coordinates import Distance
import pkg_resources
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [ ]:
# import agnpy classes
from agnpy.spectra import PowerLaw, BrokenPowerLaw
from agnpy.emission_regions import Blob
from agnpy.synchrotron import Synchrotron, ProtonSynchrotron
from agnpy.compton import SynchrotronSelfCompton
from agnpy.utils.plot import plot_sed

### 3.1. Synchrotron and Synchrotron Self-Compton emission in `agnpy`
Let us consider that the emission region is a blob of radius $R_b = 10^{16}\,\mathrm{cm}$, streaming with Lorentz factor $\Gamma=10$ along the jet axis, resulting in a Doppler boosting of $\delta_D=10$ (the jet is almost aligned with the observer's line of sight). The blob has a tangled magnetic field $B=1\,\mathrm{G}$. The galaxy hosting the active nucleus is at a distance of $d_L=10^{27}\,\mathrm{cm}$. 

The electron spectrum is described by a power law with index $-2.8$ with Lorentz factors in the range $10^2-10^7$ and a total energy content of $W_{\rm e} = 10^{48}\,{\rm erg}$.

In [ ]:
# blob attributes
R_b = 1e16 * u.cm
B = 1 * u.G
z = Distance(1e27, unit=u.cm).z
delta_D = 10
Gamma = 10

# total energy content of the electron distribution
W_e = 1e48 * u.Unit("erg")
V_b = 4 / 3 * np.pi * R_b**3

n_e = PowerLaw.from_total_energy(
    W_e,
    V_b,
    p=2.8,
    gamma_min=1e2,
    gamma_max=1e7,
    mass=m_e,
)

# emission region
blob = Blob(R_b=R_b, z=z, delta_D=delta_D, Gamma=Gamma, B=B, n_e=n_e)

In [ ]:
print(blob)

### 3.1.1. Synchrotron radiation
To initialize the object that will compute the synchrotron radiation, **we simply pass the `Blob` instance to the `Synchrotron` class**. Synchrotron self absorption can also be considered.

In [ ]:
synch = Synchrotron(blob)
synch_ssa = Synchrotron(blob, ssa=True)

Every class defining a radiative process has a `.sed_flux` function computing the SED over an array of frequencies.

In [ ]:
# let us define a grid of frequencies over which to calculate the synchrotron SED
nu_synch = np.logspace(8, 23) * u.Hz

# compute a synchrotron, and a self-absorbed synchrotron SED
sed_synch = synch.sed_flux(nu_synch)
sed_synch_ssa = synch_ssa.sed_flux(nu_synch)

In [ ]:
plot_sed(nu_synch, sed_synch, color="k", label="synch.")
plot_sed(nu_synch, sed_synch_ssa, ls="--", color="gray", label="self-absorbed synch.")
plt.show()

### 3.1.2. Synchrotron Self-Compton Radiation
Similarly to the synchrotron case, to initialize the object that will compute the synchrotron self-Compton (SSC) radiation, we simply pass the `Blob` instance to the `SynchrotronSelfCompton` class.       
We examine also the case produced by Compton scattering of a self-absorbed synchrotron spectrum.

In [ ]:
# simple ssc
ssc = SynchrotronSelfCompton(blob)
# ssc over a self-absorbed synchrotron spectrum
ssc_ssa = SynchrotronSelfCompton(blob, ssa=True)

In [ ]:
nu_ssc = np.logspace(15, 30) * u.Hz

sed_ssc = ssc.sed_flux(nu_ssc)
sed_ssc_ssa = ssc_ssa.sed_flux(nu_ssc)

In [ ]:
plot_sed(nu_ssc, sed_ssc, color="k", label="SSC")
plot_sed(nu_ssc, sed_ssc_ssa, ls="--", color="gray", label="SSC with SSA")
plt.show()

## 3.2. Validation: compare the result of `agnpy` against the literature and `JetSet`

In order to validate `agnpy` we can compare the results of its calculations against the literature and against other software.

For what concerns the literature, we choose to reproduce Figure 7.4 of [Dermer and Menon (2009)](https://ui.adsabs.harvard.edu/abs/2009herb.book.....D/abstract). The figure, reported here, depicts two complete synchrotron and SSC SEDs, generated by the same emission regions distinguished only by the different maximum Lorentz factor of their electron distributions ($\gamma_{\rm max} = 10^5$ and $\gamma_{\rm max} = 10^7$).

In [ ]:
url = "https://raw.githubusercontent.com/cosimoNigro/agnpy/master/docs/tutorials/figures/figure_7_4_dermer_2009.png"
Image(url, width=800, height=600)

In the figure, the model with $\gamma_{\rm max} = 10^7$ has the same parameter values we used in the definition of our blob. The numerical values of the SED in the reference figure are stored in the `agnpy` resources.

In [ ]:
# load the SED from the reference figure
synch_dermer = pkg_resources.resource_filename(
    "agnpy",
    "data/reference_seds/dermer_menon_2009/figure_7_4/synchrotron_gamma_max_1e7.txt",
)

ssc_dermer = pkg_resources.resource_filename(
    "agnpy", "data/reference_seds/dermer_menon_2009/figure_7_4/ssc_gamma_max_1e7.txt"
)

data_synch_dermer = np.loadtxt(synch_dermer, delimiter=",")
data_ssc_dermer = np.loadtxt(ssc_dermer, delimiter=",")

nu_synch_dermer = data_synch_dermer[:, 0] * u.Hz
sed_synch_dermer = data_synch_dermer[:, 1] * u.Unit("erg cm-2 s-1")

nu_ssc_dermer = data_ssc_dermer[:, 0] * u.Hz
sed_ssc_dermer = data_ssc_dermer[:, 1] * u.Unit("erg cm-2 s-1")

We then produce a final validation plot.

In [ ]:
plot_sed(
    nu_synch,
    sed_synch,
    color="crimson",
    label="agnpy",
)
plot_sed(nu_ssc, sed_ssc, color="crimson")

plot_sed(
    nu_synch_dermer,
    sed_synch_dermer,
    color="k",
    ls="--",
    label="Figure 7.4, Dermer et al. 2009",
)
plot_sed(nu_ssc_dermer, sed_ssc_dermer, color="k", ls="--")

plt.legend(fontsize=11)
plt.ylim([1e-13, 1e-8])
plt.show()

We get an excellent agreement, further validation examples are provided in the [agnpy release paper](https://arxiv.org/abs/2112.14573).

## 3.3. Exercises 

### 3.3.1 Equipartition
Determine if this jet is at equipartition.

### 3.3.2 Peak of the synchrotron distribution
Consider that the blob previously defined contains a `BrokenPowerLaw` of electrons with spectral indexes $p_1 = 2.1$, $p_2=3.5$, break Lorentz factor $\gamma_b=10^4$, minimum and maximum Lorentz factors $\gamma_{\rm min}=10$ and $\gamma_{\rm max}=10^6$, respectively. The normalisation is irrelevant for the sake of the exercise. Obtain the frequency where the peak of the synchrotron emission is occurring (search in the `agnpy` documentation for the `Synchrotron` class and its functions). Using the slides, or the agnpy documentation, calculate the peak of the the single-particle synchrotron emission for a particle with $\gamma=\gamma_b=10^4$. Remember that frequencies are boosted by a multiplicative factor $\delta_{\rm D}$. Are the two frequency close? What could we conclude?

## 3.4. Proton Synchrotron

Since version `0.4.0` the synchrotron radiation of protons is available in `agnpy`.
Let us define a proton particle distribution and add it to the blob.

In [ ]:
# total energy content of the electron distribution
W_p = 1e54 * u.Unit("erg")
V_b = 4 / 3 * np.pi * R_b**3

n_p = PowerLaw.from_total_energy(
    W_p,
    V_b,
    p=2.5,
    gamma_min=1e3,
    gamma_max=1e11,
    mass=m_p,
)

# re-define the emission region
blob = Blob(R_b=R_b, z=z, delta_D=delta_D, Gamma=Gamma, B=B, n_e=n_e, n_p=n_p)

In [ ]:
# the ProtonSynchrotron class works exactly as the Synchrotron class
synch = Synchrotron(blob)

psynch = ProtonSynchrotron(blob)

nu = np.logspace(8, 28, 200) * u.Hz

sed_synch = synch.sed_flux(nu)
sed_psynch = psynch.sed_flux(nu)

In [ ]:
fig, ax = plt.subplots()

plot_sed(
    nu,
    sed_synch,
    ax=ax,
    label="electron synchrotron",
)
plot_sed(
    nu,
    sed_psynch,
    ax=ax,
    color="crimson",
    label="proton synchrotron",
)

ax.set_ylim([1e-14, 1e-8])
plt.show()